In [50]:
import nrrd
import matplotlib.pyplot as plt
import numpy as np
from subtle.utils.io import write_dicoms, dicom_files
import SimpleITK as sitk
import time
import uuid
import os

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (12, 10)

def process_nrrd_img(fpath_nrrd):
    img, _ = nrrd.read(fpath_nrrd)
    img = img.transpose(2, 0, 1)
    img = np.rot90(img, axes=(1, 2), k=3)
    img = np.interp(img, (img.min(), img.max()), (0, 1000))
    return img

def generate_uid(prefix='1.2.826.0.1.3680043.10.221.'):
    entropy_src = uuid.uuid4().int
    # Convert this to an int with the maximum available digits
    avail_digits = 64 - len(prefix)
    int_val = entropy_src % (10 ** avail_digits)
    return prefix + str(int_val)

def nrrd_to_dicom(fpath_nrrd, fpath_out_dicom, study_uid, pid, snum):
    reader = sitk.ImageFileReader()
    reader.SetImageIO("NrrdImageIO")
    reader.SetFileName(fpath_nrrd)
    
    img_sitk = reader.Execute()
    img_np = sitk.GetArrayFromImage(img_sitk)
    img_np = np.interp(img_np, (img_np.min(), img_np.max()), (0, 1000))
    img_np = img_np.astype(np.uint16)
    img_sitk_sc = sitk.GetImageFromArray(img_np)
    img_sitk_sc.CopyInformation(img_sitk)
    img_sitk = img_sitk_sc
    
    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()

    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    direction = img_sitk.GetDirection()
    px_spacing = "\\".join(np.array(img_sitk.GetSpacing()[:2]).astype(str))
    sl_thick = str(img_sitk.GetSpacing()[-1])
    
    series_tag_values = [("0008|0031", modification_time),  # Series Time
                      ("0008|0021", modification_date),  # Series Date
                      ("0008|0008", "DERIVED\\SECONDARY"),  # Image Type
                      ("0020|000e", study_uid),
                      ("0010|0010", pid),
                      ("0010|0020", pid),
                      ("0008|103E", '{}_{}'.format(pid, snum)),
                      ("0020|0011", str(snum)),
                      ("0028|0030", px_spacing),
                      ("0018|0050", sl_thick),
                      # Series Instance UID
                      ("0020|0037",
                       '\\'.join(map(str, (direction[0], direction[3],
                                           direction[6],
                                           # Image Orientation (Patient)
                                           direction[1], direction[4],
                                           direction[7])))),
                      ]
    
    if not os.path.exists(fpath_out_dicom):
        os.makedirs(fpath_out_dicom)
    
    for i in range(img_sitk.GetDepth()):
        image_slice = img_sitk[:, :, i]

        for tag, value in series_tag_values:
            image_slice.SetMetaData(tag, value)
        image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d"))
        image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S"))
        image_slice.SetMetaData("0020|0032", '\\'.join(
         map(str, img_sitk.TransformIndexToPhysicalPoint((0, 0, i)))))
        image_slice.SetMetaData("0020|000D", study_uid)
        image_slice.SetMetaData("0020|0013", str(i))
        image_slice.SetMetaData("0020|1041", str(i))

        writer.SetFileName('{}/{:03d}.dcm'.format(fpath_out_dicom, i))
        writer.Execute(image_slice)

<Figure size 864x720 with 0 Axes>

In [51]:
base_path = '/home/srivathsa/projects/studies/gad/bracco/3Dcases'
patient_ids = ['1002071', '1231970', '1755456', '774962']
dcm_path = '/home/srivathsa/projects/studies/gad/bracco/dcms'
series_nums = {
    'pre': 301,
    'highdose': 401
}

for pid in patient_ids:
    study_uid = generate_uid()
    for dose_kw, snum in series_nums.items():
        nrrd_to_dicom('{}/{}_{}.nrrd'.format(base_path, pid, dose_kw), 
                      '{}/{}/{}_{}'.format(dcm_path, pid, dose_kw, snum), study_uid, pid, snum)

In [26]:
np.array(img_sitk.GetSpacing())

array([1.        , 1.        , 1.00000095])

In [36]:
img, hdr = dicom_files('/home/srivathsa/projects/studies/gad/bracco/dcms/1002071/pre_301')

In [38]:
hdr.PixelSpacing

['1.0', '1.0']

In [49]:
img_sitk.

(-121.06806182861328, -128.43019104003906, -22.183073043823242)